# Preparazione e Pre-elaborazione dei Dati
## Esercitazioni Pratiche

Questo notebook contiene esercitazioni pratiche sui tre temi principali del corso:
1. Raccolta e pulizia dei dati
2. Feature engineering
3. Tecniche di suddivisione e validazione dei dati

Attraverso esempi concreti e dataset reali, metteremo in pratica le tecniche apprese durante le lezioni teoriche.

## Configurazione dell'ambiente

Iniziamo installando e importando le librerie necessarie per le nostre esercitazioni.

In [ ]:
# Installazione delle librerie necessarie
!pip install pandas numpy matplotlib seaborn scikit-learn imbalanced-learn missingno yellowbrick

In [ ]:
# Importazione delle librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import missingno as msno
import warnings

# Configurazione del notebook
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
pd.set_option('display.max_columns', None)

# Parte 1: Raccolta e Pulizia dei Dati

In questa prima parte, ci concentreremo su:
- Caricamento di dati da diverse fonti
- Esplorazione e comprensione dei dati
- Identificazione e gestione dei valori mancanti
- Rilevamento e trattamento degli outlier
- Normalizzazione e standardizzazione
- Gestione dei dati duplicati

## 1.1 Caricamento dei dati

Iniziamo caricando un dataset reale. Utilizzeremo il dataset "House Prices" di Kaggle, che contiene informazioni sulle case vendute ad Ames, Iowa.

In [ ]:
# Scaricare il dataset
!wget https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv -O titanic.csv

In [ ]:
# Caricare il dataset
df = pd.read_csv('titanic.csv')

# Visualizzare le prime righe
print(f"Dimensioni del dataset: {df.shape}")
df.head()

## 1.2 Esplorazione e comprensione dei dati

Prima di iniziare la pulizia, è importante esplorare e comprendere i dati.

In [ ]:
# Informazioni generali sul dataset
df.info()

In [ ]:
# Statistiche descrittive
df.describe(include='all')

In [ ]:
# Visualizzare la distribuzione della variabile target (sopravvissuti)
plt.figure(figsize=(8, 6))
sns.countplot(x='Survived', data=df)
plt.title('Distribuzione dei sopravvissuti')
plt.xlabel('Sopravvissuto (1 = Sì, 0 = No)')
plt.ylabel('Conteggio')
plt.show()

In [ ]:
# Esplorare la relazione tra classe e sopravvivenza
plt.figure(figsize=(10, 6))
sns.countplot(x='Pclass', hue='Survived', data=df)
plt.title('Sopravvivenza per classe')
plt.xlabel('Classe (1 = Prima classe, 2 = Seconda classe, 3 = Terza classe)')
plt.ylabel('Conteggio')
plt.legend(title='Sopravvissuto', labels=['No', 'Sì'])
plt.show()

In [ ]:
# Esplorare la relazione tra sesso e sopravvivenza
plt.figure(figsize=(10, 6))
sns.countplot(x='Sex', hue='Survived', data=df)
plt.title('Sopravvivenza per sesso')
plt.xlabel('Sesso')
plt.ylabel('Conteggio')
plt.legend(title='Sopravvissuto', labels=['No', 'Sì'])
plt.show()

In [ ]:
# Distribuzione dell'età
plt.figure(figsize=(12, 6))
sns.histplot(df['Age'].dropna(), kde=True, bins=30)
plt.title('Distribuzione dell\'età')
plt.xlabel('Età')
plt.ylabel('Conteggio')
plt.show()

## 1.3 Identificazione e gestione dei valori mancanti

Ora analizziamo i valori mancanti nel dataset e applichiamo strategie appropriate per gestirli.

In [ ]:
# Conteggio dei valori mancanti per colonna
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Valori mancanti': missing_values,
    'Percentuale': missing_percent
})

missing_df[missing_df['Valori mancanti'] > 0].sort_values('Percentuale', ascending=False)

In [ ]:
# Visualizzazione dei valori mancanti
msno.matrix(df)
plt.title('Matrice dei valori mancanti')
plt.show()

In [ ]:
msno.heatmap(df)
plt.title('Heatmap delle correlazioni tra valori mancanti')
plt.show()

### Strategie per gestire i valori mancanti

Implementiamo diverse strategie per gestire i valori mancanti:

In [ ]:
# Creare una copia del dataframe per non modificare l'originale
df_cleaned = df.copy()

# 1. Eliminazione: rimuovere la colonna 'Cabin' (troppi valori mancanti)
df_cleaned.drop('Cabin', axis=1, inplace=True)

# 2. Imputazione: sostituire i valori mancanti in 'Age' con la mediana
df_cleaned['Age'].fillna(df_cleaned['Age'].median(), inplace=True)

# 3. Imputazione: sostituire i valori mancanti in 'Embarked' con la moda
most_common_embarked = df_cleaned['Embarked'].mode()[0]
df_cleaned['Embarked'].fillna(most_common_embarked, inplace=True)

# 4. Imputazione avanzata: sostituire i valori mancanti in 'Fare' con la mediana per classe
for pclass in [1, 2, 3]:
    median_fare = df_cleaned[df_cleaned['Pclass'] == pclass]['Fare'].median()
    df_cleaned.loc[(df_cleaned['Fare'].isnull()) & (df_cleaned['Pclass'] == pclass), 'Fare'] = median_fare

# Verificare che non ci siano più valori mancanti
df_cleaned.isnull().sum()

## 1.4 Rilevamento e trattamento degli outlier

Ora identifichiamo e gestiamo gli outlier nei dati numerici.

In [ ]:
# Visualizzare la distribuzione delle variabili numeriche con box plot
numeric_cols = ['Age', 'Fare', 'SibSp', 'Parch']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    sns.boxplot(y=df_cleaned[col], ax=axes[i])
    axes[i].set_title(f'Box Plot di {col}')
    axes[i].set_ylabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# Identificare gli outlier usando il metodo IQR
def identify_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Identificare gli outlier nella tariffa (Fare)
fare_outliers, lower_bound, upper_bound = identify_outliers(df_cleaned, 'Fare')
print(f"Numero di outlier in 'Fare': {len(fare_outliers)}")
print(f"Limite inferiore: {lower_bound:.2f}, Limite superiore: {upper_bound:.2f}")
print("\nEsempi di outlier:")
fare_outliers[['PassengerId', 'Pclass', 'Fare']].head()

In [ ]:
# Trattare gli outlier con diverse strategie
df_no_outliers = df_cleaned.copy()

# 1. Rimozione degli outlier
# df_no_outliers = df_no_outliers[(df_no_outliers['Fare'] >= lower_bound) & (df_no_outliers['Fare'] <= upper_bound)]

# 2. Capping (winsorization)
df_no_outliers['Fare_capped'] = df_no_outliers['Fare'].clip(lower=lower_bound, upper=upper_bound)

# 3. Trasformazione logaritmica
df_no_outliers['Fare_log'] = np.log1p(df_no_outliers['Fare'])  # log(1+x) per gestire valori zero

# Visualizzare l'effetto delle trasformazioni
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(df_no_outliers['Fare'], kde=True, ax=axes[0])
axes[0].set_title('Distribuzione originale di Fare')
axes[0].set_xlabel('Fare')

sns.histplot(df_no_outliers['Fare_capped'], kde=True, ax=axes[1])
axes[1].set_title('Distribuzione di Fare dopo capping')
axes[1].set_xlabel('Fare_capped')

sns.histplot(df_no_outliers['Fare_log'], kde=True, ax=axes[2])
axes[2].set_title('Distribuzione di Fare dopo trasformazione logaritmica')
axes[2].set_xlabel('Fare_log')

plt.tight_layout()
plt.show()

## 1.5 Normalizzazione e standardizzazione

Applichiamo tecniche di scaling ai dati numerici.

In [ ]:
# Selezionare le feature numeriche
numeric_features = ['Age', 'Fare_log', 'SibSp', 'Parch']
X_numeric = df_no_outliers[numeric_features].copy()

# Applicare diverse tecniche di scaling
# 1. Standardizzazione (Z-score)
scaler = StandardScaler()
X_standardized = pd.DataFrame(
    scaler.fit_transform(X_numeric),
    columns=[f"{col}_standardized" for col in X_numeric.columns]
)

# 2. Min-Max Scaling
min_max_scaler = MinMaxScaler()
X_min_max = pd.DataFrame(
    min_max_scaler.fit_transform(X_numeric),
    columns=[f"{col}_min_max" for col in X_numeric.columns]
)

# Combinare i risultati
scaling_results = pd.concat([X_numeric, X_standardized, X_min_max], axis=1)
scaling_results.head()

In [ ]:
# Visualizzare l'effetto dello scaling
fig, axes = plt.subplots(3, 1, figsize=(14, 15))

# Dati originali
scaling_results[numeric_features].boxplot(ax=axes[0])
axes[0].set_title('Dati originali')

# Dati standardizzati
standardized_cols = [f"{col}_standardized" for col in numeric_features]
scaling_results[standardized_cols].boxplot(ax=axes[1])
axes[1].set_title('Dati standardizzati (Z-score)')

# Dati normalizzati (Min-Max)
min_max_cols = [f"{col}_min_max" for col in numeric_features]
scaling_results[min_max_cols].boxplot(ax=axes[2])
axes[2].set_title('Dati normalizzati (Min-Max)')

plt.tight_layout()
plt.show()

## 1.6 Gestione dei dati duplicati

Identifichiamo e gestiamo eventuali dati duplicati nel dataset.

In [ ]:
# Verificare se ci sono duplicati nel dataset
duplicates = df_no_outliers.duplicated()
print(f"Numero di righe duplicate: {duplicates.sum()}")

# Se ci sono duplicati, visualizzarli
if duplicates.sum() > 0:
    df_no_outliers[df_no_outliers.duplicated(keep=False)].sort_values(by=df_no_outliers.columns.tolist())

In [ ]:
# Verificare duplicati parziali (basati su sottoinsiemi di colonne)
# Ad esempio, cerchiamo passeggeri con stesso nome, età e classe
subset_cols = ['Name', 'Age', 'Pclass', 'Sex']
partial_duplicates = df_no_outliers.duplicated(subset=subset_cols, keep=False)
print(f"Numero di righe con duplicati parziali: {partial_duplicates.sum()}")

# Visualizzare i duplicati parziali
if partial_duplicates.sum() > 0:
    df_no_outliers[partial_duplicates].sort_values(by=subset_cols)

In [ ]:
# Rimuovere i duplicati (se necessario)
df_no_duplicates = df_no_outliers.drop_duplicates()
print(f"Dimensioni del dataset originale: {df_no_outliers.shape}")
print(f"Dimensioni del dataset senza duplicati: {df_no_duplicates.shape}")

# Parte 2: Feature Engineering

In questa seconda parte, ci concentreremo su:
- Trasformazione delle variabili
- Creazione di nuove feature
- Selezione delle feature
- Riduzione della dimensionalità
- Encoding di variabili categoriche

## 2.1 Trasformazione delle variabili

Applichiamo diverse trasformazioni alle variabili esistenti.

In [ ]:
# Creare una copia del dataframe pulito
df_fe = df_no_duplicates.copy()

# Trasformazioni matematiche
# 1. Trasformazione logaritmica (già applicata a Fare)
# 2. Trasformazione radice quadrata
df_fe['Fare_sqrt'] = np.sqrt(df_fe['Fare'])

# Visualizzare l'effetto delle trasformazioni
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(df_fe['Fare'], kde=True, ax=axes[0])
axes[0].set_title('Distribuzione originale di Fare')
axes[0].set_xlabel('Fare')

sns.histplot(df_fe['Fare_log'], kde=True, ax=axes[1])
axes[1].set_title('Trasformazione logaritmica')
axes[1].set_xlabel('Fare_log')

sns.histplot(df_fe['Fare_sqrt'], kde=True, ax=axes[2])
axes[2].set_title('Trasformazione radice quadrata')
axes[2].set_xlabel('Fare_sqrt')

plt.tight_layout()
plt.show()

## 2.2 Creazione di nuove feature

Creiamo nuove feature basate su quelle esistenti.

In [ ]:
# 1. Feature basate sul dominio
# Dimensione della famiglia (numero di fratelli/sorelle/coniugi + genitori/figli + se stessi)
df_fe['FamilySize'] = df_fe['SibSp'] + df_fe['Parch'] + 1

# 2. Feature binaria: viaggia da solo?
df_fe['IsAlone'] = (df_fe['FamilySize'] == 1).astype(int)

# 3. Estrarre il titolo dal nome
df_fe['Title'] = df_fe['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Visualizzare la distribuzione dei titoli
print(df_fe['Title'].value_counts())

# 4. Raggruppare i titoli meno comuni
title_mapping = {
    'Mr': 'Mr',
    'Miss': 'Miss',
    'Mrs': 'Mrs',
    'Master': 'Master',
    'Dr': 'Rare',
    'Rev': 'Rare',
    'Col': 'Rare',
    'Major': 'Rare',
    'Mlle': 'Miss',
    'Countess': 'Rare',
    'Ms': 'Miss',
    'Lady': 'Rare',
    'Jonkheer': 'Rare',
    'Don': 'Rare',
    'Dona': 'Rare',
    'Mme': 'Mrs',
    'Capt': 'Rare',
    'Sir': 'Rare'
}
df_fe['Title'] = df_fe['Title'].map(title_mapping)

# 5. Tariffa per persona
df_fe['FarePerPerson'] = df_fe['Fare'] / df_fe['FamilySize']

# Visualizzare le nuove feature
df_fe[['PassengerId', 'Name', 'Title', 'SibSp', 'Parch', 'FamilySize', 'IsAlone', 'Fare', 'FarePerPerson']].head()

In [ ]:
# Analizzare la relazione tra le nuove feature e la sopravvivenza
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Sopravvivenza per dimensione della famiglia
sns.countplot(x='FamilySize', hue='Survived', data=df_fe, ax=axes[0, 0])
axes[0, 0].set_title('Sopravvivenza per dimensione della famiglia')
axes[0, 0].set_xlabel('Dimensione della famiglia')
axes[0, 0].set_ylabel('Conteggio')
axes[0, 0].legend(title='Sopravvissuto', labels=['No', 'Sì'])

# Sopravvivenza per viaggiatori solitari vs. in famiglia
sns.countplot(x='IsAlone', hue='Survived', data=df_fe, ax=axes[0, 1])
axes[0, 1].set_title('Sopravvivenza per viaggiatori solitari vs. in famiglia')
axes[0, 1].set_xlabel('Viaggia da solo (1 = Sì, 0 = No)')
axes[0, 1].set_ylabel('Conteggio')
axes[0, 1].legend(title='Sopravvissuto', labels=['No', 'Sì'])

# Sopravvivenza per titolo
sns.countplot(x='Title', hue='Survived', data=df_fe, ax=axes[1, 0])
axes[1, 0].set_title('Sopravvivenza per titolo')
axes[1, 0].set_xlabel('Titolo')
axes[1, 0].set_ylabel('Conteggio')
axes[1, 0].legend(title='Sopravvissuto', labels=['No', 'Sì'])

# Distribuzione della tariffa per persona in base alla sopravvivenza
sns.boxplot(x='Survived', y='FarePerPerson', data=df_fe, ax=axes[1, 1])
axes[1, 1].set_title('Tariffa per persona in base alla sopravvivenza')
axes[1, 1].set_xlabel('Sopravvissuto (1 = Sì, 0 = No)')
axes[1, 1].set_ylabel('Tariffa per persona')

plt.tight_layout()
plt.show()

## 2.3 Encoding di variabili categoriche

Applichiamo diverse tecniche di encoding alle variabili categoriche.

In [ ]:
# Identificare le variabili categoriche
categorical_features = ['Sex', 'Embarked', 'Title']

# 1. Label Encoding
df_encoded = df_fe.copy()
label_encoders = {}

for feature in categorical_features:
    le = LabelEncoder()
    df_encoded[f'{feature}_label'] = le.fit_transform(df_encoded[feature])
    label_encoders[feature] = le
    
    # Mostrare la mappatura
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"Label Encoding per {feature}:")
    print(mapping)
    print()

# Visualizzare il risultato
df_encoded[[*categorical_features, *[f'{f}_label' for f in categorical_features]]].head()

In [ ]:
# 2. One-Hot Encoding
df_onehot = df_fe.copy()

# Applicare One-Hot Encoding
df_onehot = pd.get_dummies(df_onehot, columns=categorical_features, drop_first=True)

# Visualizzare il risultato
print(f"Dimensioni originali: {df_fe.shape}")
print(f"Dimensioni dopo One-Hot Encoding: {df_onehot.shape}")
df_onehot.head()

In [ ]:
# 3. Target Encoding
df_target_encoded = df_fe.copy()

for feature in categorical_features:
    # Calcolare la media della variabile target per ogni categoria
    target_means = df_target_encoded.groupby(feature)['Survived'].mean()
    
    # Applicare l'encoding
    df_target_encoded[f'{feature}_target'] = df_target_encoded[feature].map(target_means)
    
    # Mostrare la mappatura
    print(f"Target Encoding per {feature}:")
    print(target_means)
    print()

# Visualizzare il risultato
df_target_encoded[[*categorical_features, *[f'{f}_target' for f in categorical_features]]].head()

## 2.4 Selezione delle feature

Applichiamo diverse tecniche per selezionare le feature più rilevanti.

In [ ]:
# Preparare i dati per la selezione delle feature
# Utilizziamo il dataframe con one-hot encoding
X = df_onehot.drop(['Survived', 'Name', 'Ticket', 'PassengerId'], axis=1)
y = df_onehot['Survived']

print(f"Feature disponibili: {X.shape[1]}")
print(X.columns.tolist())

In [ ]:
# 1. Filter Method: Correlazione con la variabile target
correlation = X.corrwith(y).abs().sort_values(ascending=False)

plt.figure(figsize=(12, 8))
correlation.plot(kind='bar')
plt.title('Correlazione assoluta con la sopravvivenza')
plt.xlabel('Feature')
plt.ylabel('Correlazione assoluta')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Selezionare le top 10 feature
top_corr_features = correlation.nlargest(10).index.tolist()
print("Top 10 feature per correlazione:")
print(top_corr_features)

In [ ]:
# 2. Filter Method: ANOVA F-value
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X, y)

# Ottenere i punteggi e i p-value
scores = pd.DataFrame({
    'Feature': X.columns,
    'Score': selector.scores_,
    'P-value': selector.pvalues_
})

# Ordinare per punteggio
scores = scores.sort_values('Score', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Score', y='Feature', data=scores.head(10))
plt.title('Top 10 feature per F-value (ANOVA)')
plt.tight_layout()
plt.show()

# Selezionare le top 10 feature
top_anova_features = scores.head(10)['Feature'].tolist()
print("Top 10 feature per F-value:")
print(top_anova_features)

In [ ]:
# 3. Embedded Method: Random Forest Feature Importance
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# Ottenere l'importanza delle feature
importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
})

# Ordinare per importanza
importances = importances.sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importances.head(10))
plt.title('Top 10 feature per importanza (Random Forest)')
plt.tight_layout()
plt.show()

# Selezionare le top 10 feature
top_rf_features = importances.head(10)['Feature'].tolist()
print("Top 10 feature per importanza (Random Forest):")
print(top_rf_features)

In [ ]:
# 4. Wrapper Method: Recursive Feature Elimination (RFE)
model = LogisticRegression(max_iter=1000, random_state=42)
rfe = RFE(estimator=model, n_features_to_select=10, step=1)
rfe.fit(X, y)

# Ottenere il ranking delle feature
rfe_ranking = pd.DataFrame({
    'Feature': X.columns,
    'Ranking': rfe.ranking_,
    'Selected': rfe.support_
})

# Ordinare per ranking
rfe_ranking = rfe_ranking.sort_values('Ranking')

# Visualizzare le feature selezionate
selected_features = rfe_ranking[rfe_ranking['Selected']]
print("Feature selezionate da RFE:")
print(selected_features['Feature'].tolist())

## 2.5 Riduzione della dimensionalità

Applichiamo tecniche di riduzione della dimensionalità per visualizzare e comprimere i dati.

In [ ]:
# Standardizzare i dati prima della PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Applicare PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Visualizzare la varianza spiegata
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

plt.figure(figsize=(12, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, label='Varianza individuale')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Varianza cumulativa')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% varianza spiegata')
plt.xlabel('Numero di componenti')
plt.ylabel('Proporzione di varianza spiegata')
plt.title('Varianza spiegata dalle componenti principali')
plt.legend()
plt.tight_layout()
plt.show()

# Determinare il numero di componenti per spiegare il 95% della varianza
n_components = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Numero di componenti necessarie per spiegare il 95% della varianza: {n_components}")

In [ ]:
# Applicare PCA con 2 componenti per la visualizzazione
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

# Creare un dataframe con le componenti principali
pca_df = pd.DataFrame({
    'PC1': X_pca_2d[:, 0],
    'PC2': X_pca_2d[:, 1],
    'Survived': y
})

# Visualizzare i dati nel nuovo spazio bidimensionale
plt.figure(figsize=(10, 8))
sns.scatterplot(x='PC1', y='PC2', hue='Survived', data=pca_df, palette='viridis', alpha=0.8)
plt.title('PCA: Visualizzazione dei dati in 2D')
plt.xlabel('Prima componente principale')
plt.ylabel('Seconda componente principale')
plt.legend(title='Sopravvissuto', labels=['No', 'Sì'])
plt.tight_layout()
plt.show()

In [ ]:
# Analizzare il contributo delle feature alle componenti principali
components = pd.DataFrame(
    pca.components_,
    columns=X.columns
)

plt.figure(figsize=(14, 10))
sns.heatmap(components.iloc[:5], annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Contributo delle feature alle prime 5 componenti principali')
plt.xlabel('Feature')
plt.ylabel('Componente principale')
plt.tight_layout()
plt.show()

# Parte 3: Tecniche di Suddivisione e Validazione dei Dati

In questa terza parte, ci concentreremo su:
- Train-test split
- Validazione incrociata (cross-validation)
- Stratificazione
- Metriche di valutazione
- Gestione dello sbilanciamento delle classi

## 3.1 Train-Test Split

Applichiamo la suddivisione base in training e test set.

In [ ]:
# Preparare i dati
# Utilizziamo le feature selezionate dal Random Forest
X_selected = X[top_rf_features]
y = df_onehot['Survived']

# Suddivisione base (70% training, 30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=42)

print(f"Dimensioni del training set: {X_train.shape}")
print(f"Dimensioni del test set: {X_test.shape}")

In [ ]:
# Verificare la distribuzione della variabile target nei subset
print("Distribuzione della variabile target nel dataset completo:")
print(y.value_counts(normalize=True))
print("\nDistribuzione della variabile target nel training set:")
print(y_train.value_counts(normalize=True))
print("\nDistribuzione della variabile target nel test set:")
print(y_test.value_counts(normalize=True))

In [ ]:
# Addestrare un modello sul training set e valutarlo sul test set
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fare predizioni sul test set
y_pred = model.predict(X_test)

# Valutare le performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

In [ ]:
# Visualizzare la matrice di confusione
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matrice di confusione')
plt.xlabel('Predetto')
plt.ylabel('Reale')
plt.xticks([0.5, 1.5], ['Non sopravvissuto (0)', 'Sopravvissuto (1)'])
plt.yticks([0.5, 1.5], ['Non sopravvissuto (0)', 'Sopravvissuto (1)'])
plt.tight_layout()
plt.show()

## 3.2 Validazione Incrociata (Cross-Validation)

Applichiamo la validazione incrociata per ottenere una stima più robusta delle performance.

In [ ]:
# K-Fold Cross-Validation
model = RandomForestClassifier(n_estimators=100, random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Calcolare i punteggi
cv_scores = cross_val_score(model, X_selected, y, cv=kf, scoring='accuracy')

print(f"Punteggi per fold: {cv_scores}")
print(f"Punteggio medio: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

In [ ]:
# Calcolare diverse metriche con cross-validation
from sklearn.model_selection import cross_validate

scoring = ['accuracy', 'precision', 'recall', 'f1']
cv_results = cross_validate(model, X_selected, y, cv=kf, scoring=scoring)

# Visualizzare i risultati
for metric in scoring:
    scores = cv_results[f'test_{metric}']
    print(f"{metric.capitalize()}: {scores.mean():.4f} (±{scores.std():.4f})")

In [ ]:
# Visualizzare i risultati della cross-validation
cv_results_df = pd.DataFrame({
    'Fold': range(1, kf.n_splits + 1),
    'Accuracy': cv_results['test_accuracy'],
    'Precision': cv_results['test_precision'],
    'Recall': cv_results['test_recall'],
    'F1': cv_results['test_f1']
})

cv_results_melted = pd.melt(cv_results_df, id_vars=['Fold'], var_name='Metric', value_name='Score')

plt.figure(figsize=(12, 6))
sns.barplot(x='Fold', y='Score', hue='Metric', data=cv_results_melted)
plt.title('Risultati della Cross-Validation per fold')
plt.xlabel('Fold')
plt.ylabel('Punteggio')
plt.ylim(0, 1)
plt.legend(title='Metrica')
plt.tight_layout()
plt.show()

## 3.3 Stratificazione

Applichiamo la stratificazione per mantenere la stessa distribuzione delle classi in tutti i subset.

In [ ]:
# Stratified K-Fold Cross-Validation
model = RandomForestClassifier(n_estimators=100, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calcolare i punteggi
stratified_cv_scores = cross_val_score(model, X_selected, y, cv=skf, scoring='accuracy')

print(f"Punteggi per fold (stratificati): {stratified_cv_scores}")
print(f"Punteggio medio (stratificato): {stratified_cv_scores.mean():.4f} (±{stratified_cv_scores.std():.4f})")

In [ ]:
# Confrontare K-Fold standard con Stratified K-Fold
comparison_df = pd.DataFrame({
    'Fold': range(1, kf.n_splits + 1),
    'K-Fold standard': cv_scores,
    'Stratified K-Fold': stratified_cv_scores
})

comparison_melted = pd.melt(comparison_df, id_vars=['Fold'], var_name='Metodo', value_name='Accuracy')

plt.figure(figsize=(12, 6))
sns.barplot(x='Fold', y='Accuracy', hue='Metodo', data=comparison_melted)
plt.title('Confronto tra K-Fold standard e Stratified K-Fold')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.ylim(0.7, 0.9)  # Adattare in base ai risultati
plt.legend(title='Metodo')
plt.tight_layout()
plt.show()

In [ ]:
# Stratified Train-Test Split
X_train_strat, X_test_strat, y_train_strat, y_test_strat = train_test_split(
    X_selected, y, test_size=0.3, stratify=y, random_state=42)

print("Distribuzione della variabile target nel dataset completo:")
print(y.value_counts(normalize=True))
print("\nDistribuzione della variabile target nel training set stratificato:")
print(y_train_strat.value_counts(normalize=True))
print("\nDistribuzione della variabile target nel test set stratificato:")
print(y_test_strat.value_counts(normalize=True))

## 3.4 Metriche di Valutazione

Approfondiamo le diverse metriche di valutazione per problemi di classificazione.

In [ ]:
# Addestrare un modello sul training set stratificato
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_strat, y_train_strat)

# Fare predizioni sul test set
y_pred = model.predict(X_test_strat)
y_pred_proba = model.predict_proba(X_test_strat)[:, 1]  # Probabilità della classe positiva

In [ ]:
# Calcolare diverse metriche
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, log_loss, confusion_matrix, classification_report)

accuracy = accuracy_score(y_test_strat, y_pred)
precision = precision_score(y_test_strat, y_pred)
recall = recall_score(y_test_strat, y_pred)
f1 = f1_score(y_test_strat, y_pred)
auc = roc_auc_score(y_test_strat, y_pred_proba)
loss = log_loss(y_test_strat, y_pred_proba)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {auc:.4f}")
print(f"Log Loss: {loss:.4f}")

print("\nClassification Report:")
print(classification_report(y_test_strat, y_pred))

In [ ]:
# Visualizzare la curva ROC
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(y_test_strat, y_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualizzare la curva Precision-Recall
from sklearn.metrics import precision_recall_curve, average_precision_score

precision_curve, recall_curve, _ = precision_recall_curve(y_test_strat, y_pred_proba)
avg_precision = average_precision_score(y_test_strat, y_pred_proba)

plt.figure(figsize=(10, 8))
plt.plot(recall_curve, precision_curve, color='blue', lw=2, 
         label=f'Precision-Recall curve (AP = {avg_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3.5 Gestione dello Sbilanciamento delle Classi

Applichiamo tecniche per gestire lo sbilanciamento delle classi.

In [ ]:
# Verificare lo sbilanciamento delle classi
class_counts = y.value_counts()
print("Distribuzione delle classi:")
print(class_counts)
print(f"Rapporto di sbilanciamento: {class_counts[0] / class_counts[1]:.2f}")

plt.figure(figsize=(8, 6))
sns.countplot(x=y)
plt.title('Distribuzione delle classi')
plt.xlabel('Classe')
plt.ylabel('Conteggio')
plt.xticks([0, 1], ['Non sopravvissuto (0)', 'Sopravvissuto (1)'])
plt.tight_layout()
plt.show()

In [ ]:
# Creare un dataset artificialmente sbilanciato per dimostrare le tecniche
# (Il dataset Titanic è già leggermente sbilanciato, ma creiamo uno sbilanciamento più marcato)

# Selezionare casualmente un sottoinsieme della classe minoritaria (sopravvissuti)
minority_indices = y[y == 1].index
np.random.seed(42)
remove_indices = np.random.choice(minority_indices, size=int(len(minority_indices) * 0.7), replace=False)

# Creare il dataset sbilanciato
X_imbalanced = X_selected.drop(remove_indices)
y_imbalanced = y.drop(remove_indices)

# Verificare lo sbilanciamento
imbalanced_counts = y_imbalanced.value_counts()
print("Distribuzione delle classi nel dataset sbilanciato:")
print(imbalanced_counts)
print(f"Rapporto di sbilanciamento: {imbalanced_counts[0] / imbalanced_counts[1]:.2f}")

plt.figure(figsize=(8, 6))
sns.countplot(x=y_imbalanced)
plt.title('Distribuzione delle classi nel dataset sbilanciato')
plt.xlabel('Classe')
plt.ylabel('Conteggio')
plt.xticks([0, 1], ['Non sopravvissuto (0)', 'Sopravvissuto (1)'])
plt.tight_layout()
plt.show()

In [ ]:
# Suddividere il dataset sbilanciato
X_train_imb, X_test_imb, y_train_imb, y_test_imb = train_test_split(
    X_imbalanced, y_imbalanced, test_size=0.3, stratify=y_imbalanced, random_state=42)

# Addestrare un modello sul dataset sbilanciato
model_imb = RandomForestClassifier(n_estimators=100, random_state=42)
model_imb.fit(X_train_imb, y_train_imb)

# Fare predizioni
y_pred_imb = model_imb.predict(X_test_imb)

# Valutare le performance
print("Performance sul dataset sbilanciato:")
print(f"Accuracy: {accuracy_score(y_test_imb, y_pred_imb):.4f}")
print(f"Precision: {precision_score(y_test_imb, y_pred_imb):.4f}")
print(f"Recall: {recall_score(y_test_imb, y_pred_imb):.4f}")
print(f"F1-score: {f1_score(y_test_imb, y_pred_imb):.4f}")

# Visualizzare la matrice di confusione
cm_imb = confusion_matrix(y_test_imb, y_pred_imb)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_imb, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matrice di confusione (dataset sbilanciato)')
plt.xlabel('Predetto')
plt.ylabel('Reale')
plt.xticks([0.5, 1.5], ['Non sopravvissuto (0)', 'Sopravvissuto (1)'])
plt.yticks([0.5, 1.5], ['Non sopravvissuto (0)', 'Sopravvissuto (1)'])
plt.tight_layout()
plt.show()

In [ ]:
# 1. Random Undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_imb, y_train_imb)

print("Distribuzione delle classi dopo Random Undersampling:")
print(pd.Series(y_train_rus).value_counts())

# Addestrare un modello
model_rus = RandomForestClassifier(n_estimators=100, random_state=42)
model_rus.fit(X_train_rus, y_train_rus)

# Fare predizioni
y_pred_rus = model_rus.predict(X_test_imb)

# Valutare le performance
print("\nPerformance dopo Random Undersampling:")
print(f"Accuracy: {accuracy_score(y_test_imb, y_pred_rus):.4f}")
print(f"Precision: {precision_score(y_test_imb, y_pred_rus):.4f}")
print(f"Recall: {recall_score(y_test_imb, y_pred_rus):.4f}")
print(f"F1-score: {f1_score(y_test_imb, y_pred_rus):.4f}")

In [ ]:
# 2. SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_imb, y_train_imb)

print("Distribuzione delle classi dopo SMOTE:")
print(pd.Series(y_train_smote).value_counts())

# Addestrare un modello
model_smote = RandomForestClassifier(n_estimators=100, random_state=42)
model_smote.fit(X_train_smote, y_train_smote)

# Fare predizioni
y_pred_smote = model_smote.predict(X_test_imb)

# Valutare le performance
print("\nPerformance dopo SMOTE:")
print(f"Accuracy: {accuracy_score(y_test_imb, y_pred_smote):.4f}")
print(f"Precision: {precision_score(y_test_imb, y_pred_smote):.4f}")
print(f"Recall: {recall_score(y_test_imb, y_pred_smote):.4f}")
print(f"F1-score: {f1_score(y_test_imb, y_pred_smote):.4f}")

In [ ]:
# 3. Pesi delle classi
# Calcolare i pesi inversamente proporzionali alla frequenza delle classi
class_weights = {0: 1, 1: imbalanced_counts[0] / imbalanced_counts[1]}
print(f"Pesi delle classi: {class_weights}")

# Addestrare un modello con pesi delle classi
model_weighted = RandomForestClassifier(n_estimators=100, class_weight=class_weights, random_state=42)
model_weighted.fit(X_train_imb, y_train_imb)

# Fare predizioni
y_pred_weighted = model_weighted.predict(X_test_imb)

# Valutare le performance
print("\nPerformance con pesi delle classi:")
print(f"Accuracy: {accuracy_score(y_test_imb, y_pred_weighted):.4f}")
print(f"Precision: {precision_score(y_test_imb, y_pred_weighted):.4f}")
print(f"Recall: {recall_score(y_test_imb, y_pred_weighted):.4f}")
print(f"F1-score: {f1_score(y_test_imb, y_pred_weighted):.4f}")

In [ ]:
# Confrontare le diverse tecniche
techniques = ['Senza trattamento', 'Random Undersampling', 'SMOTE', 'Pesi delle classi']
accuracies = [
    accuracy_score(y_test_imb, y_pred_imb),
    accuracy_score(y_test_imb, y_pred_rus),
    accuracy_score(y_test_imb, y_pred_smote),
    accuracy_score(y_test_imb, y_pred_weighted)
]
precisions = [
    precision_score(y_test_imb, y_pred_imb),
    precision_score(y_test_imb, y_pred_rus),
    precision_score(y_test_imb, y_pred_smote),
    precision_score(y_test_imb, y_pred_weighted)
]
recalls = [
    recall_score(y_test_imb, y_pred_imb),
    recall_score(y_test_imb, y_pred_rus),
    recall_score(y_test_imb, y_pred_smote),
    recall_score(y_test_imb, y_pred_weighted)
]
f1_scores = [
    f1_score(y_test_imb, y_pred_imb),
    f1_score(y_test_imb, y_pred_rus),
    f1_score(y_test_imb, y_pred_smote),
    f1_score(y_test_imb, y_pred_weighted)
]

# Creare un dataframe con i risultati
results_df = pd.DataFrame({
    'Tecnica': techniques,
    'Accuracy': accuracies,
    'Precision': precisions,
    'Recall': recalls,
    'F1-score': f1_scores
})

# Visualizzare i risultati
results_melted = pd.melt(results_df, id_vars=['Tecnica'], var_name='Metrica', value_name='Valore')

plt.figure(figsize=(14, 8))
sns.barplot(x='Tecnica', y='Valore', hue='Metrica', data=results_melted)
plt.title('Confronto delle tecniche per gestire lo sbilanciamento delle classi')
plt.xlabel('Tecnica')
plt.ylabel('Valore')
plt.ylim(0, 1)
plt.legend(title='Metrica')
plt.tight_layout()
plt.show()

# Conclusioni

In questo notebook abbiamo esplorato le principali tecniche di preparazione e pre-elaborazione dei dati, coprendo i tre temi principali del corso:

1. **Raccolta e pulizia dei dati**:
   - Caricamento e esplorazione dei dati
   - Identificazione e gestione dei valori mancanti
   - Rilevamento e trattamento degli outlier
   - Normalizzazione e standardizzazione
   - Gestione dei dati duplicati

2. **Feature engineering**:
   - Trasformazione delle variabili
   - Creazione di nuove feature
   - Encoding di variabili categoriche
   - Selezione delle feature
   - Riduzione della dimensionalità

3. **Tecniche di suddivisione e validazione dei dati**:
   - Train-test split
   - Validazione incrociata (cross-validation)
   - Stratificazione
   - Metriche di valutazione
   - Gestione dello sbilanciamento delle classi

Queste tecniche sono fondamentali per qualsiasi progetto di machine learning e data science, e spesso hanno un impatto maggiore sulle performance finali rispetto alla scelta dell'algoritmo.

# Esercizi Proposti

1. Carica un altro dataset a tua scelta (ad esempio da Kaggle o UCI Machine Learning Repository) e applica le tecniche di pulizia dei dati viste in questo notebook.

2. Crea nuove feature basate sul dominio specifico del dataset scelto e valuta il loro impatto sulle performance di un modello.

3. Confronta diverse tecniche di encoding per le variabili categoriche e identifica quale funziona meglio per il tuo dataset.

4. Implementa una pipeline completa di pre-elaborazione dei dati utilizzando `sklearn.pipeline.Pipeline` e `sklearn.compose.ColumnTransformer`.

5. Sperimenta con diverse tecniche di gestione dello sbilanciamento delle classi e analizza il loro impatto sulle diverse metriche di valutazione.